# Backtest Your Short-Position Algorithm

In this activity, you’ll backtest an algorithm to determine both the changes to the overall portfolio values and the daily return and cumulative return metrics.

## Instructions:

1. Review the provided code. This includes the `import` statements for the required libraries, the creation of the Pandas DataFrame from the `vnq.csv` file in the `Resources` folder, and the code that creates and visualizes the short-position DMAC trading algorithm.

2. Set an `initial_capital` variable to `100000` to represent the starting value ($100,000) of your simulated portfolio. Be sure to create this variable as a floating point number.

3. Set a `share_size` variable to negative 500 shares (-500).
    > **Note:** Remember that you’re using a short-position strategy. The objective is to enter the trade by selling shares at a high price before buying them back, or exiting the trade, at a lower price.

4. Create a new column named “Position” by multiplying the `share_size` value by the values in the “Signal” column.

5. Create a new column named “Entry/Exit Position” by calling the `diff` function on the “Position” column.

6. Create a new column named “Portfolio Holdings” by multiplying the “Close” prices of VNQ by the values in the “Position” column.

7. Create a new column named “Portfolio Cash” by subtracting the cumulative sum of the trades that the “Entry/Exit Position” column indicates from the `initial_capital` value.

8. Create a new column named “Portfolio Total” by adding the values of the “Portfolio Cash” and “Portfolio Holdings” columns.

9. Create a new column named “Portfolio Daily Returns” by calling the `pct_change` function on the “Portfolio Total” column.

10. Create a new column named “Portfolio Cumulative Returns” by calling the `cum_prod` function on the “Portfolio Daily Returns” column.

11. Use the `hvplot` function to plot the short-position DMAC trading strategy against its backtesting results.

12. Review the resulting overlay plot. Then answer the following question: Does it appear that this algorithm suits a short-position trading strategy?



## Step 1: Review the provided code. This includes the `import` statements for the required libraries, the creation of the Pandas DataFrame from the `vnq.csv` file in the `Resources` folder, and the code that creates and visualizes the short-position DMAC trading algorithm.

In [1]:
# Imports
import numpy as np
import pandas as pd
import hvplot.pandas
from pathlib import Path

### Read the CSV file in from the resources folder and create the Pandas DataFrame

In [2]:
# Read the vnq.csv file located in the Resources folder into a Pandas DataFrame
# Set the Date column as the DateTimeIndex
vnq_df = pd.read_csv(
    Path("../Resources/vnq.csv"),
    index_col="Date",
    parse_dates=True, 
    infer_datetime_format=True
)

# Review the DataFrame
vnq_df.head()

C:\Users\pablo\AppData\Local\Temp\ipykernel_48776\1590594049.py:3: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  vnq_df = pd.read_csv(
C:\Users\pablo\AppData\Local\Temp\ipykernel_48776\1590594049.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  vnq_df = pd.read_csv(


,Open,High,Low,Close,Volume
Date,,,,,
2007-01-03,77.339996,77.709999,76.400002,44.707417,525700
2007-01-04,77.230003,77.260002,76.629997,44.765377,166100
2007-01-05,77.099998,77.099998,75.750000,43.936401,143700
2007-01-08,76.000000,76.000000,75.430000,44.000164,87000
2007-01-09,75.730003,76.959999,75.730003,44.556675,136300


### Generate a Dual Moving Average Crossover Trading Signal

In [3]:
# Filter the Date index and close columns
signals_df = vnq_df.loc[:,["Close"]]

# Set the short window and long window
short_window = 50
long_window = 100

# Generate the short and long moving averages (50 and 100 days, respectively)
signals_df['SMA50'] = signals_df['Close'].rolling(window=short_window).mean()
signals_df['SMA100'] = signals_df['Close'].rolling(window=long_window).mean()
signals_df['Signal'] = 0.0

# Generate the trading signal 0 or 1,
# where 1 is when the SMA50 is less than the SMA100
# where 0 is when the SMA50 is greater than than the SMA100
signals_df['Signal'][short_window:] = np.where(
    signals_df['SMA50'][short_window:] < signals_df['SMA100'][short_window:], 1.0, 0.0
)

# Calculate the points in time at which a position should be taken, 1 or -1
signals_df['Entry/Exit'] = signals_df['Signal'].diff()

# Review the DataFrame
signals_df.tail(10)

,Close,SMA50,SMA100,Signal,Entry/Exit
Date,,,,,
2009-12-17,29.639492,28.167839,27.217638,0.0,0.0
2009-12-18,29.835062,28.211270,27.294345,0.0,0.0
2009-12-21,30.239697,28.258072,27.367152,0.0,0.0
2009-12-22,30.556675,28.312024,27.443329,0.0,0.0
2009-12-23,30.920845,28.381082,27.519007,0.0,0.0
2009-12-24,31.367579,28.438977,27.586397,0.0,0.0
2009-12-28,31.517859,28.504734,27.645874,0.0,0.0
2009-12-29,31.046522,28.575901,27.702575,0.0,0.0
2009-12-30,31.094339,28.636155,27.745596,0.0,0.0


### Plot Entry and Exit Points of Dual Moving Average Crossover Trading Strategy

In [4]:
# Visualize entry positions relative to close price
entry = signals_df[signals_df['Entry/Exit'] == 1.0]['Close'].hvplot.scatter(
    color='purple',
    marker='^',
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400)

# Visualize exit positions relative to close price
exit = signals_df[signals_df['Entry/Exit'] == -1.0]['Close'].hvplot.scatter(
    color='orange',
    marker='v',
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400)

# Visualize the close price for the investment
security_close = signals_df[['Close']].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400)

# Visualize the moving averages
moving_avgs = signals_df[['SMA50', 'SMA100']].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400)

# Overlay the plots
entry_exit_plot = security_close * moving_avgs * entry * exit
entry_exit_plot.opts(
    title="VNQ - Short-Position Dual Moving Average Trading Algorithm"
)

:Overlay
   .Curve.I          :Curve   [Date]   (Close)
   .NdOverlay.I      :NdOverlay   [Variable]
      :Curve   [Date]   (value)
   .Scatter.Close.I  :Scatter   [Date]   (Close)
   .Scatter.Close.II :Scatter   [Date]   (Close)

## Step 2: Set an `initial_capital` variable to `100000` to represent the starting value (100,000) of your simulated portfolio. Be sure to create this variable as a floating point number.

In [5]:
# Set the initial_capital to 100000
initial_capital = float(100000)

## Step 3: Set a `share_size` variable to negative 500 shares (-500).

In [6]:
# Set the share_size to -500
# In a short-position strategy, shares are sold before they are bought 
share_size = -500

## Step 4: Create a new column named “Position” by multiplying the `share_size` value by the values in the “Signal” column.

In [7]:
# Create a column named "Position" by multiplying the share_size by the Signal
# Sell a position (-500 shares) when the dual moving average crossover Signal equals 1 (SMA50 is less than SMA100)
# Buy a position (500 shares) when the dual moving average crossover Signal equals 0 (SMA50 is greater than SMA100)
signals_df["Position"] = share_size * signals_df["Signal"]

# Review the DataFrame
signals_df.tail()

,Close,SMA50,SMA100,Signal,Entry/Exit,Position
Date,,,,,,
2009-12-24,31.367579,28.438977,27.586397,0.0,0.0,-0.0
2009-12-28,31.517859,28.504734,27.645874,0.0,0.0,-0.0
2009-12-29,31.046522,28.575901,27.702575,0.0,0.0,-0.0
2009-12-30,31.094339,28.636155,27.745596,0.0,0.0,-0.0
2009-12-31,30.561541,28.696138,27.789032,0.0,0.0,-0.0


## Step 5: Create a new column named “Entry/Exit Position” by calling the `diff` function on the “Position” column.

In [8]:
# Find the points in time where a 500 share position is bought or sold
signals_df["Entry/Exit Position"] = signals_df["Position"].diff()

# Review the DataFrame
signals_df.tail()

,Close,SMA50,SMA100,Signal,Entry/Exit,Position,Entry/Exit Position
Date,,,,,,,
2009-12-24,31.367579,28.438977,27.586397,0.0,0.0,-0.0,0.0
2009-12-28,31.517859,28.504734,27.645874,0.0,0.0,-0.0,0.0
2009-12-29,31.046522,28.575901,27.702575,0.0,0.0,-0.0,0.0
2009-12-30,31.094339,28.636155,27.745596,0.0,0.0,-0.0,0.0
2009-12-31,30.561541,28.696138,27.789032,0.0,0.0,-0.0,0.0


## Step 6: Create a new column named “Portfolio Holdings” by multiplying the “Close” prices of VNQ by the values in the “Position” column.


In [9]:
# Create a Portfolio Holdings column by multiplying the Close price by the Position
signals_df["Portfolio Holdings"] = signals_df["Close"] * signals_df["Position"]


# Review the DataFrame
signals_df.tail()

,Close,SMA50,SMA100,Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings
Date,,,,,,,,
2009-12-24,31.367579,28.438977,27.586397,0.0,0.0,-0.0,0.0,-0.0
2009-12-28,31.517859,28.504734,27.645874,0.0,0.0,-0.0,0.0,-0.0
2009-12-29,31.046522,28.575901,27.702575,0.0,0.0,-0.0,0.0,-0.0
2009-12-30,31.094339,28.636155,27.745596,0.0,0.0,-0.0,0.0,-0.0
2009-12-31,30.561541,28.696138,27.789032,0.0,0.0,-0.0,0.0,-0.0


## Step 7: Create a new column named “Portfolio Cash” by subtracting the cumulative sum of the trades that the “Entry/Exit Position” column indicates from the `initial_capital` value.

In [10]:
# To calculate Portfolio Cash, subtract the cumulative sum of the trade cost/proceeds from the initial_capital
# The trade cost proceeds are calculated by multiplying the Close price by Entry/Exit Position
signals_df["Portfolio Cash"] = (
    initial_capital - (signals_df["Close"] * signals_df["Entry/Exit Position"]).cumsum()
)

# Review the DataFrame
signals_df.tail()

,Close,SMA50,SMA100,Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash
Date,,,,,,,,,
2009-12-24,31.367579,28.438977,27.586397,0.0,0.0,-0.0,0.0,-0.0,106587.2295
2009-12-28,31.517859,28.504734,27.645874,0.0,0.0,-0.0,0.0,-0.0,106587.2295
2009-12-29,31.046522,28.575901,27.702575,0.0,0.0,-0.0,0.0,-0.0,106587.2295
2009-12-30,31.094339,28.636155,27.745596,0.0,0.0,-0.0,0.0,-0.0,106587.2295
2009-12-31,30.561541,28.696138,27.789032,0.0,0.0,-0.0,0.0,-0.0,106587.2295


## Step 8: Create a new column named “Portfolio Total” by adding the values of the “Portfolio Cash” and “Portfolio Holdings” columns.

In [11]:
# Calculate the Portfolio Total value by adding Portfolio Cash and Portfolio Holdings 
signals_df["Portfolio Total"] = signals_df["Portfolio Cash"] + signals_df["Portfolio Holdings"]

# Review the DataFrame
signals_df.tail()

,Close,SMA50,SMA100,Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total
Date,,,,,,,,,,
2009-12-24,31.367579,28.438977,27.586397,0.0,0.0,-0.0,0.0,-0.0,106587.2295,106587.2295
2009-12-28,31.517859,28.504734,27.645874,0.0,0.0,-0.0,0.0,-0.0,106587.2295,106587.2295
2009-12-29,31.046522,28.575901,27.702575,0.0,0.0,-0.0,0.0,-0.0,106587.2295,106587.2295
2009-12-30,31.094339,28.636155,27.745596,0.0,0.0,-0.0,0.0,-0.0,106587.2295,106587.2295
2009-12-31,30.561541,28.696138,27.789032,0.0,0.0,-0.0,0.0,-0.0,106587.2295,106587.2295


## Step 9: Create a new column named “Portfolio Daily Returns” by calling the `pct_change` function on the “Portfolio Total” column.

In [12]:
# Calculate the Portfolio Daily Returns based on the Portfolio Total
signals_df["Portfolio Daily Returns"] = signals_df["Portfolio Total"].pct_change()

# Review the DataFrame
signals_df.tail()

,Close,SMA50,SMA100,Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total,Portfolio Daily Returns
Date,,,,,,,,,,,
2009-12-24,31.367579,28.438977,27.586397,0.0,0.0,-0.0,0.0,-0.0,106587.2295,106587.2295,0.0
2009-12-28,31.517859,28.504734,27.645874,0.0,0.0,-0.0,0.0,-0.0,106587.2295,106587.2295,0.0
2009-12-29,31.046522,28.575901,27.702575,0.0,0.0,-0.0,0.0,-0.0,106587.2295,106587.2295,0.0
2009-12-30,31.094339,28.636155,27.745596,0.0,0.0,-0.0,0.0,-0.0,106587.2295,106587.2295,0.0
2009-12-31,30.561541,28.696138,27.789032,0.0,0.0,-0.0,0.0,-0.0,106587.2295,106587.2295,0.0


## Step 10: Create a new column named “Portfolio Cumulative Returns” by calling the `cum_prod` function on the “Portfolio Daily Returns” column.

In [13]:
# Calculate the Portfolio Cumulative Returns based on the Portfolio Daily Returns
signals_df["Portfolio Cumulative Returns"] = (
    1 + signals_df["Portfolio Daily Returns"]
).cumprod() - 1

# Review the DataFrame
signals_df.tail()

,Close,SMA50,SMA100,Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total,Portfolio Daily Returns,Portfolio Cumulative Returns
Date,,,,,,,,,,,,
2009-12-24,31.367579,28.438977,27.586397,0.0,0.0,-0.0,0.0,-0.0,106587.2295,106587.2295,0.0,0.065872
2009-12-28,31.517859,28.504734,27.645874,0.0,0.0,-0.0,0.0,-0.0,106587.2295,106587.2295,0.0,0.065872
2009-12-29,31.046522,28.575901,27.702575,0.0,0.0,-0.0,0.0,-0.0,106587.2295,106587.2295,0.0,0.065872
2009-12-30,31.094339,28.636155,27.745596,0.0,0.0,-0.0,0.0,-0.0,106587.2295,106587.2295,0.0,0.065872
2009-12-31,30.561541,28.696138,27.789032,0.0,0.0,-0.0,0.0,-0.0,106587.2295,106587.2295,0.0,0.065872


## Step 11: Use the `hvplot` function to plot the short-position DMAC trading strategy against its backtesting results.

In [14]:
# Visualize the entry positions relative to the Portfolio Total
entry = signals_df[signals_df["Entry/Exit"] == 1.0]["Portfolio Total"].hvplot.scatter(
    color='purple',
    marker='^',
    legend=False, 
    ylabel="Total Portfolio Value", 
    width=1000, 
    height=400
)

# Visualize the exit positions relative to the Portfolio Total
exit = signals_df[signals_df["Entry/Exit"] == -1.0]["Portfolio Total"].hvplot.scatter(
    color='orange',
    marker='v',
    legend=False, 
    ylabel="Total Portfolio Value", 
    width=1000, 
    height=400
)

# Visualize Portfolio Total for the investment
total_portfolio_value = signals_df[["Portfolio Total"]].hvplot(
    line_color="lightgray", 
    ylabel="Total Portfolio Value", 
    width=1000, 
    height=400
)

# Overlay the entry, exit and total_portfolio_value plots
portfolio_entry_exit_plot = total_portfolio_value * entry * exit
portfolio_entry_exit_plot.opts(
    title="VAQ Short-Position Algorithm - Total Portfolio Value",
    yformatter='%.0f'
)

:Overlay
   .Curve.I                    :Curve   [Date]   (Portfolio Total)
   .Scatter.Portfolio_Total.I  :Scatter   [Date]   (Portfolio Total)
   .Scatter.Portfolio_Total.II :Scatter   [Date]   (Portfolio Total)

## Step 12: Answer the following question:

**Question:** Does it appear that this algorithm suits a short-position trading strategy?
    
**Answer:** Although the value of the portfolio did fall below the initial investment of 100,000,  the drop in the stock price in 2008 and 2009 benefited the algorithm's short-position strategy and the portfolio did make money.